<a href="https://colab.research.google.com/github/gibranfp/CursoDatosMasivosI/blob/main/notebooks/3d_lsh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsqueda del vecino más cercano aproximado mediante funciones _hash_ sensibles a la localidad
En esta libreta se realiza un buscador del vecino más cercano aproximado usando funciones _hash_ sensibles a la localidad (LSH). Especificamente, se define la familia LSH basada  en distribuciones $p$-estables para distancias $\ell_1$ y $\ell_2$ y otra familia para la distancia angular.

In [1]:
from os import listdir
from os.path import isfile, join
import struct
import os 

import numpy as np

import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
from skimage import io, transform

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download(['punkt','averaged_perceptron_tagger','wordnet'])
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.corpus.reader.wordnet import NOUN, VERB, ADV, ADJ

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Conjunto de datos
Para evaluar el buscador vamos usar el conjunto de vectores SIFT [ANN_SIFT10K](http://corpus-texmex.irisa.fr/) del grupo TEXMEX, el cual descargamos y extraemos.

In [2]:
!wget -q ftp://ftp.irisa.fr/local/texmex/corpus/siftsmall.tar.gz
!tar xvzf siftsmall.tar.gz

siftsmall/
siftsmall/siftsmall_base.fvecs
siftsmall/siftsmall_groundtruth.ivecs
siftsmall/siftsmall_learn.fvecs
siftsmall/siftsmall_query.fvecs


Definimos una función para leer los vectores de un archivo `.fvecs`.

In [3]:
import struct
import os 

def lee_fvecs(ruta):
  with open(ruta, 'rb') as f:
    d = struct.unpack('i', f.read(4))[0]
    n = f.seek(0, os.SEEK_END) // (4 + 4 * d)
    f.seek(0)
    vecs = np.zeros((n, d))
    for i in range(n):
      f.read(4)
      vecs[i] = struct.unpack('f' * d, f.read(d * 4))
  
  return vecs 

Leemos el conjunto de vectores base y consulta.

In [4]:
base = lee_fvecs('siftsmall/siftsmall_base.fvecs')
consultas = lee_fvecs('siftsmall/siftsmall_query.fvecs')

print('Base: {0} Consultas: {1}'.format(base.shape, consultas.shape))

Base: (10000, 128) Consultas: (100, 128)


Definimos una función para leer los vectores más cercanos reales (_groundtruth_) de un archivo `.ivecs`

In [5]:
def lee_ivecs(ruta):
  with open(ruta, 'rb') as f:
    d = struct.unpack('i', f.read(4))[0]
    n = f.seek(0, os.SEEK_END) // (4 + 4 * d)
    f.seek(0)
    vecs = np.zeros((n, d), dtype=np.int)
    for i in range(n):
      f.read(4)
      vecs[i] = struct.unpack('i' * d, f.read(d * 4))
  
  return vecs 

Leemos estos vectores.

In [6]:
gt = lee_ivecs('siftsmall/siftsmall_groundtruth.ivecs')
print('Groundtruth: {0}'.format(gt.shape))

Groundtruth: (100, 100)


## Distancias $\ell_1$ y $\ell_2$.
Definimos nuestra clase de tabla _hash_ con una familia de funciones basada en distribuciones $s$-estables. En esta familia se elige aleatoriamente una proyección de $\mathbb{R}^d$ sobre una línea, se desplaza por $b$ y se corta en segmentos de tamaño $w$, esto es,
        $$
        h_{\mathbf{a},b} = \left\lfloor  \frac{\mathbf{a} \cdot \mathbf{x} + b}{w} \right\rfloor
        $$
donde $b \in [0, w)$

 * Si $\mathbf{a}$ se muestrea de una distribución normal se obtiene una familia LSH para distancia $\ell_2$.\newline
 * Si $\mathbf{a}$ se muestrea de una distribución de Cauchy se obtiene una familia LSH para distancia $\ell_1$

In [7]:
class TablaLpLSH:
  def __init__(self, n_cubetas, t_tupla, dim, width=4, norma = 'l2'):
    self.n_cubetas = n_cubetas
    self.tabla = [[] for i in range(n_cubetas)]
    self.t_tupla = t_tupla
    self.dim = dim
    self.w = width

    if norma == 'l2':
      self.Amat = np.random.standard_normal((t_tupla, dim))
    elif norma == 'l1':
      self.Amat = np.random.standard_cauchy((t_tupla, dim))

    self.bvec = np.random.uniform(low=0, high=self.w, size=t_tupla)
    self.a = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.b = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.primo = 4294967291

  def __repr__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas)]
    return "<TablaHash :%s >" % ('\n'.join(contenido))

  def __str__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas) if self.tabla[i]]
    return '\n'.join(contenido)

  def sl(self, x, i):
    return (self.h(x) + i) % self.n_cubetas

  def h(self, x):
    return x % self.primo

  def lphash(self, x):
    prod = np.floor((self.Amat @ x.T + self.bvec) / self.w).astype(np.uint)
    return np.sum(self.a * prod, dtype=np.ulonglong), np.sum(self.b * prod, dtype=np.ulonglong)
     
  def insertar(self, x, ident):
    lph, v2 = self.lphash(x)

    llena = True
    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        self.tabla[cubeta].append(lph)
        self.tabla[cubeta].append([ident])
        llena = False
        break
      elif self.tabla[cubeta][0] == lph:
        self.tabla[cubeta][1].append(ident)
        llena = False
        break

    if llena:
      print('¡Error, tabla llena!')

  def buscar(self, x):
    mh, v2 = self.lphash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        return []
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1]
        
    return []

Instanciamos las tablas.

In [8]:
n_tablas = 30
dim = base.shape[1]
tablas = [TablaLpLSH(2**14, 3, dim, 4.0, 'l2') for _ in range(n_tablas)]

Insertamos los vectores en cada tabla.

In [9]:
for i,x in enumerate(base):
  for t in range(n_tablas):
    tablas[t].insertar(x, i)

Realizamos la búsqueda de los vectores de consulta y recuperamos los vectores más similares del conjunto base.

In [10]:
vecs = []
for i,q in enumerate(consultas):
  dc = []
  for t in range(n_tablas):
      dc.extend(tablas[t].buscar(q))
  vecs.append(set(dc))

Calculamos la distancia euclidiana entre cada vector de consulta y sus correspondientes vectores recuperados y los ordenamos por distancia.

In [11]:
def distancia_euclidiana(x, y):   
  return np.sqrt(np.sum((x - y)**2))

def fuerza_bruta(ds, qs, fd):
  medidas = np.zeros(ds.shape[0])
  for i,x in enumerate(ds):
    medidas[i] = fd(qs, x)

  return np.sort(medidas), np.argsort(medidas)

dists = []
orden = []
for i,q in enumerate(consultas):
  ld = list(vecs[i])
  if ld:
    m,o = fuerza_bruta(base[ld], q, distancia_euclidiana)
    dists.append(m)
    orden.append([ld[e] for e in o])
  else:
    dists.append([])
    orden.append([])

Extraemos los vecinos más cercanos encontrados por LSH y los reales y los comparamos.

In [12]:
vmc_lsh = [o[0] if o else -1 for o in orden]
vmc_real = [g[0] for g in gt]
correcto = [vmc_lsh[i] == vmc_real[i] for i in range(len(vmc_lsh))]
print('Promedio encontrados = {0}'.format(np.mean(correcto)))

Promedio encontrados = 0.73


## Distancia angular
Definimos una clase de tabla LSH para distancia angular $ 1 - \theta(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})$  basada en la siguiente familia 
$$
h_\mathbf{v}(\mathbf{x}^{(i)}) = signo(\mathbf{v} \cdot \mathbf{x}^{(i)})
$$

donde $\mathbf{v} \in \mathbb{R}^d$ es un vector aleatorio de tamaño unitario y

 $$
\theta(\mathbf{x}^{(i)}, \mathbf{x}^{(j)}) = \arccos{\left(\frac{\mathbf{x}^{(i)} \cdot \mathbf{x}^{(j)}}{\lVert \mathbf{x}^{(i)}\rVert \cdot \lVert {\mathbf{x}^{(j)}}\rVert}\right)}
$$

La probabilidad de que cualquier par de vectores $(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})$ tenga un valor idéntico para esta familia es
$$
Pr[h_\mathbf{v}(\mathbf{x}^{(i)}) = h_\mathbf{v}(\mathbf{x}^{(j)}] = 1 - \frac{\theta(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})}{\pi}
$$


In [13]:
class TablaCos:
  def __init__(self, n_cubetas, t_tupla, dim):
    self.n_cubetas = n_cubetas
    self.tabla = [[] for i in range(n_cubetas)]
    self.t_tupla = t_tupla
    self.dim = dim

    self.Amat = np.random.standard_normal((t_tupla, dim))
    self.Amat /= np.linalg.norm(self.Amat, axis=1).reshape(-1, 1)
    self.a = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.b = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.primo = 4294967291

  def __repr__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas)]
    return "<TablaHash :%s >" % ('\n'.join(contenido))

  def __str__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas) if self.tabla[i]]
    return '\n'.join(contenido)

  def sl(self, x, i):
    return (self.h(x) + i) % self.n_cubetas

  def h(self, x):
    return x % self.primo

  def coshash(self, x):
    sign = np.sign(self.Amat @ x.T).astype(np.uint)
    return np.sum(self.a * sign, dtype=np.ulonglong), np.sum(self.b * sign, dtype=np.ulonglong)
     
  def insertar(self, x, ident):
    ch, v2 = self.coshash(x)

    llena = True
    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        self.tabla[cubeta].append(ch)
        self.tabla[cubeta].append([ident])
        llena = False
        break
      elif self.tabla[cubeta][0] == ch:
        self.tabla[cubeta][1].append(ident)
        llena = False
        break

    if llena:
      print('¡Error, tabla llena!')

  def buscar(self, x):
    ch, v2 = self.coshash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        return []
      elif self.tabla[cubeta][0] == ch:
        return self.tabla[cubeta][1]
        
    return []

Instanciamos nuestras tablas para la distancia angular

In [14]:
n_tablas_cos = 30
tablas_cos = [TablaCos(2**14, 3, dim) for _ in range(n_tablas_cos)]

Insertamos todos los vectores en cada tabla

In [15]:
for i,x in enumerate(base):
  for t in range(n_tablas_cos):
    tablas_cos[t].insertar(x, i)

Realizamos la búsqueda de todos vectores de consulta en cada tabla.

In [16]:
vecs_cos = []
for i,q in enumerate(consultas):
  dc_cos = []
  for t in range(n_tablas_cos):
      dc_cos.extend(tablas_cos[t].buscar(q))
  vecs_cos.append(set(dc_cos))

Definimos una función para calcular la similitud coseno, definida por 

$$
S_{C}(\mathbf{x}^{(i)}, \mathbf{x}^{(j)}) = \frac{\mathbf{x}^{(i)} \cdot \mathbf{x}^{(j)}}{\lVert \mathbf{x}^{(i)}\rVert \cdot \lVert \mathbf{x}^{(j)} \rVert} 
$$

Calculamos la distancia coseno entre cada consulta y los vectores recuperados de las tablas, los cuales se ordenan por su similitud con el vector de consulta.

In [17]:
def similitud_coseno(x, y):
  pnorma = (np.sqrt(x @ x) * np.sqrt(y @ y))

  if pnorma > 0:
    return (x @ y) / pnorma
  else: 
    return np.nan 

def fuerza_bruta_cos(ds, qs, fd):
  medidas = np.zeros(ds.shape[0])
  for i,x in enumerate(ds):
    medidas[i] = fd(qs, x)

  return np.sort(medidas)[::-1], np.argsort(medidas)[::-1]

sims_cos = []
orden_cos = []
for i,q in enumerate(consultas):
  ldc = list(vecs_cos[i])
  if ldc:
    mc,oc = fuerza_bruta_cos(base[ldc], q, similitud_coseno)
    sims_cos.append(mc)
    orden_cos.append([ldc[e] for e in oc])
  else:
    sims_cos.append([])
    orden_cos.append([])

Comparamos los vecinos más cercanos encontrados por LSH y los reales.

In [18]:
vmc_lshcos = [o[0] if o else -1 for o in orden_cos]
vmc_real = [g[0] for g in gt]
correctos_cos = [vmc_lshcos[i] == vmc_real[i] for i in range(len(vmc_lshcos))]
print('Promedio encontrados = {0}'.format(np.mean(correctos_cos)))

Promedio encontrados = 0.98


## Ejercicio
 * Compara el desempeño de los algoritmos usando distintos hiperparámetros
 * Usa otro conjunto de datos para evaluar los algoritmos